# Setting
* work team 등록 필요

In [1]:
from sagemaker.predictor import RealTimePredictor
from sagemaker.predictor import json_serializer, json_deserializer
from sagemaker import get_execution_role

role = get_execution_role()
endpoint = 'MyWorkflowFromScratch4-2020-06-28-09-42-37'
bucket = 'sagemaker-us-east-1-233037139193'
a2i_prefix = 'mbp3/review'
out_path = 's3://{}/{}'.format(bucket, a2i_prefix)

predictor = RealTimePredictor(endpoint=endpoint, content_type='application/json',
                              serializer=json_serializer, deserializer=json_deserializer)

In [3]:
import boto3

WORKTEAM = 'arn:aws:sagemaker:us-east-1:233037139193:workteam/private-crowd/test-team'

sm = boto3.client('sagemaker')
a2i = boto3.client('sagemaker-a2i-runtime')
s3 = boto3.client('s3')

flowName = 'test-flow2'
taskUIName = 'test-task-ui3'

# Task UI creation

https://github.com/aws-samples/amazon-a2i-sample-task-uis

In [1]:
template = r"""
<script src="https://assets.crowd.aws/crowd-html-elements.js"></script>

<crowd-form>
  <crowd-image-classifier
    name="crowd-image-classifier"
    src="{{ task.input.taskObject | grant_read_access }}"
    header="Please select the correct category for this image"
    categories="['Zero', 'One', 'Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'don\'t know']"
  >
    <full-instructions header="Classification Instructions">
      <p>Read the task carefully and inspect the image.</p>
      <p>Choose the appropriate label that best suits the image.</p>
    </full-instructions>

    <short-instructions>
      <p>Read the task carefully and inspect the image.</p>
      <p>Choose the appropriate label that best suits the image.</p>
    </short-instructions>
  </crowd-image-classifier>
</crowd-form>
"""

In [5]:
humanTaskUiResponse = sm.create_human_task_ui(HumanTaskUiName=taskUIName,
                                            UiTemplate={'Content': template})
humanTaskUiArn = humanTaskUiResponse['HumanTaskUiArn']
#humanTaskUiArn = 'arn:aws:sagemaker:us-east-1:233037139193:human-task-ui/test-task-ui3'
print(humanTaskUiArn)

arn:aws:sagemaker:us-east-1:233037139193:human-task-ui/test-task-ui3


# Create human work flow

In [6]:
create_wf_response = sm.create_flow_definition(FlowDefinitionName=flowName,
                                              RoleArn = role,
                                              HumanLoopConfig = {
                                                  "WorkteamArn": WORKTEAM,
                                                  "HumanTaskUiArn": humanTaskUiArn,
                                                  "TaskCount": 1,
                                                  "TaskDescription": "MNIST labeling",
                                                  "TaskTitle": "MNIST test"
                                              },
                                              OutputConfig={
                                                  "S3OutputPath": out_path
                                              })
flowDefinitionArn = create_wf_response['FlowDefinitionArn']
#flowDefinitionArn = 'arn:aws:sagemaker:us-east-1:233037139193:flow-definition/test-flow5'

In [7]:
describeFlowDefinition = sm.describe_flow_definition(FlowDefinitionName=flowName)
for x in range(60):
    print(describeFlowDefinition['FlowDefinitionStatus'])
    if(describeFlowDefinition['FlowDefinitionStatus'] == 'Active'):
        print("Flow definition is active")
        break
    time.sleep(2)

Active
Flow definition is active


# Human loop start

In [8]:
import numpy as np
sample_data = np.loadtxt('../00_Basics/test_sample.csv', delimiter=',')

In [9]:
test_data = sample_data[31:60, 1:]
test_label = sample_data[31:60, 0]
predictions = predictor.predict(test_data)

In [10]:
import uuid
import json
import matplotlib.image

In [1]:
human_loops = []
ACC_THRESHOLD = .90
for i in range(len(predictions['predictions'])):
    prediction = np.argmax(predictions['predictions'][i])

    if(predictions['predictions'][i][prediction] < ACC_THRESHOLD):
        img = test_data[i] * 255.0
        img_reshape = img.reshape((28,28))
        matplotlib.image.imsave('a2itest{}.png'.format(i), img_reshape)
        s3.upload_file('a2itest{}.png'.format(i), bucket, a2i_prefix + '/' + 'a2itest{}.png'.format(i))
        humanLoopName = str(uuid.uuid4())
        inputContent = {
            "initialValue": test_label[i],
            "taskObject": out_path + '/a2itest{}.png'.format(i)
        }
        print(out_path + '/a2itest{}.png'.format(i))
        start_loop = a2i.start_human_loop(HumanLoopName=humanLoopName,
                                          FlowDefinitionArn=flowDefinitionArn,
                                          HumanLoopInput={
                                              "InputContent": json.dumps(inputContent)
                                          })
        human_loops.append(humanLoopName)
        print(json.dumps(inputContent))

NameError: name 'predictions' is not defined

In [13]:
completed_human_loops = []
for human_loop_name in human_loops:
    resp = a2i.describe_human_loop(HumanLoopName=human_loop_name)
    print(f'HumanLoop Name: {human_loop_name}')
    print(f'HumanLoop Status: {resp["HumanLoopStatus"]}')
    print(f'HumanLoop Output Destination: {resp["HumanLoopOutput"]}')
    print('\n')
    
    if resp["HumanLoopStatus"] == "Completed":
        completed_human_loops.append(resp)

HumanLoop Name: 7ff639f8-58cc-4661-9812-3284e8f272a2
HumanLoop Status: Completed
HumanLoop Output Destination: {'OutputS3Uri': 's3://sagemaker-us-east-1-233037139193/mbp3/review/test-flow2/2020/07/02/02/24/04/7ff639f8-58cc-4661-9812-3284e8f272a2/output.json'}


HumanLoop Name: ab33e56f-6543-47d1-9e32-fac56698abcd
HumanLoop Status: Completed
HumanLoop Output Destination: {'OutputS3Uri': 's3://sagemaker-us-east-1-233037139193/mbp3/review/test-flow2/2020/07/02/02/24/05/ab33e56f-6543-47d1-9e32-fac56698abcd/output.json'}


HumanLoop Name: 7244bad3-8a8f-4316-84c9-4d07d78ce346
HumanLoop Status: Completed
HumanLoop Output Destination: {'OutputS3Uri': 's3://sagemaker-us-east-1-233037139193/mbp3/review/test-flow2/2020/07/02/02/24/05/7244bad3-8a8f-4316-84c9-4d07d78ce346/output.json'}




# Human loop result

In [14]:
import pprint
import re

pp = pprint.PrettyPrinter(indent=4)

fixed_items = []

for resp in completed_human_loops:
    split_string = re.split('s3://' +  bucket + '/', resp['HumanLoopOutput']['OutputS3Uri'])
    output_bucket_key = split_string[1]

    response = s3.get_object(Bucket=bucket, Key=output_bucket_key)
    content = response['Body'].read().decode('utf-8')
    json_output = json.loads(content)
    print(json_output)

    input_content = json_output['inputContent']
    human_answer = json_output['humanAnswers'][0]['answerContent']
    fixed_item = {'input_content': input_content, 'human_answer': human_answer}
    fixed_items.append(fixed_item)

{'flowDefinitionArn': 'arn:aws:sagemaker:us-east-1:233037139193:flow-definition/test-flow2', 'humanAnswers': [{'answerContent': {'crowd-image-classifier': {'label': 'Six'}}, 'submissionTime': '2020-07-02T02:29:09.133Z', 'workerId': '22185cd53a22e995', 'workerMetadata': {'identityData': {'identityProviderType': 'Cognito', 'issuer': 'https://cognito-idp.us-east-1.amazonaws.com/us-east-1_MW7OYSCWx', 'sub': 'e2635a34-8482-4a34-9c11-31b05731233f'}}}], 'humanLoopName': '7ff639f8-58cc-4661-9812-3284e8f272a2', 'inputContent': {'initialValue': 6, 'taskObject': 's3://sagemaker-us-east-1-233037139193/mbp3/review/a2itest2.png'}}
{'flowDefinitionArn': 'arn:aws:sagemaker:us-east-1:233037139193:flow-definition/test-flow2', 'humanAnswers': [{'answerContent': {'crowd-image-classifier': {'label': 'Seven'}}, 'submissionTime': '2020-07-02T02:29:12.728Z', 'workerId': '22185cd53a22e995', 'workerMetadata': {'identityData': {'identityProviderType': 'Cognito', 'issuer': 'https://cognito-idp.us-east-1.amazonaws

In [15]:
import pandas as pd
df_fixed_items = pd.DataFrame(fixed_items)

In [16]:
df_fixed_items

,human_answer,input_content
0,{'crowd-image-classifier': {'label': 'Six'}},"{'initialValue': 6, 'taskObject': 's3://sagema..."
1,{'crowd-image-classifier': {'label': 'Seven'}},"{'initialValue': 7, 'taskObject': 's3://sagema..."
2,{'crowd-image-classifier': {'label': 'Nine'}},"{'initialValue': 9, 'taskObject': 's3://sagema..."
